In [112]:
import pandas as pd
import numpy as np
import nltk
import urllib
import requests
import string
import os
import matplotlib.pyplot as plt
import csv
import re
import unicodedata

from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split



In [101]:
news_df = pd.read_csv(os.path.join('news-data', 'uci-news-aggregator.csv'), header=0, index_col=0, dtype=str)
print('Read .csv file from local copy of dataset.')

Read .csv file from local copy of dataset.


In [102]:
news_df.head()

,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
ID,,,,,,,
1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [78]:
#news_df.isnull().sum() THERE ARE 2 NULL PUBLISHERS

In [79]:
def remove_stopwords(s1):
    s1 = ' '.join([unicodedata.normalize("NFKD", word)
                   for word in s1.split()
                   if word not in nltk.corpus.stopwords.words('english')])
    return s1

In [80]:
def remove_punctuation(s1):
    """
    Returns s1 without punctuation that is not word-internal (e.g., hyphens, apostrophes)
    removed.
    """
    # TODO: optimise with pandas' str replace
    
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    s1 = re.sub('\b\W+',' ',s1)
    s1 = re.sub('\W+\b',' ',s1)
    #s1 = re.sub('(\b\W+)|(\W+\b)',' ',s1)
    return s1

In [103]:
news_df['TEXT']  = news_df['TITLE'].str.lower()
#title = unicodedata.normalize("NFKD", title)

In [113]:
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(news_df['TEXT'])

encoder = OneHotEncoder()
labels = encoder.fit_transform(news_df[['CATEGORY']])

In [114]:
# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

In [115]:
x_train.shape, x_text.shape, y_train.shape, y_test.shape

NameError: name 'x_text' is not defined

In [74]:
%%time

a.apply(remove_stopwords)

KeyboardInterrupt: 